In [19]:
import sys
sys.path.append("C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\Working_attributes")
from Imports import * 

In [20]:
files_list = ['2024-10-15_Invest_nv_SUEK.xlsx', '2024-10-15_Invest_nv_EUROCHEM.xlsx']
old_data_SU = pd.read_excel(Output_path+files_list[0])
old_data_ECH = pd.read_excel(Output_path+files_list[1])
old_data = pd.concat([old_data_SU,old_data_ECH]).reset_index(drop=True)

In [21]:
base = 'RISKACCESS'
base = 'xxwh'
new_data = export_from_WHWEEK(f'''select * 
from {base}."unifiedOcp"
where "sourceTable" = 'sapBwInvestmentReport'
and "reportDate" = TO_DATE('30.09.24', 'DD.MM.YY')
--fetch first 10 rows only''')
new_data.shape

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:176: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)


(35743, 40)

In [30]:
data_OCP_work = new_data[['holding','buCode','salesUnitsId','ocpSegment','counterpartySegment','counterpartyCode','effectiveCurrency','volumeNotionalCurrency','maturityDateTo']]
data_OCP_work.maturityDateTo = data_OCP_work.maturityDateTo.astype(str).apply(lambda x: '-'.join([x[:-3],'01']))
data_OCP_work.maturityDateTo = pd.to_datetime(data_OCP_work.maturityDateTo)

C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_17164\2796183029.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_OCP_work.maturityDateTo = data_OCP_work.maturityDateTo.astype(str).apply(lambda x: '-'.join([x[:-3],'01']))
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_17164\2796183029.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_OCP_work.maturityDateTo = pd.to_datetime(data_OCP_work.maturityDateTo)


In [31]:
# merge sum to dates
data_OCP_work.counterpartyCode = data_OCP_work.counterpartyCode.astype(str)
old_data.Контрагент = old_data.Контрагент.astype(str)
data_OCP_work.ocpSegment = data_OCP_work.ocpSegment.fillna('External')

leaft_list = ['holding','Comp_segment','Comp_code','Контрагент','currency','КалендГод/Месяц']
ver_cols = ['Версия_035', 'Версия_045','Версия_075', 'Версия_085']
right_list = ['holding','ocpSegment','buCode','counterpartyCode','effectiveCurrency','maturityDateTo']

old_data['Amount_abs'] = old_data['Amount'].abs()
old_data_pivot = old_data.groupby(as_index=False,dropna=False, by=leaft_list)[['Amount_abs']+ver_cols].agg(sum)
data_OCP_work['Amount_abs'] = data_OCP_work['volumeNotionalCurrency'].abs()
data_OCP_work_pivot = data_OCP_work.groupby(as_index=False,dropna=False, by=right_list)['Amount_abs'].agg(sum)
# merge
merge_data = old_data_pivot.merge(data_OCP_work_pivot, how='outer', left_on=leaft_list, right_on=right_list, suffixes=['_old','_OCP'])
# merge_data[merge_data.Amount_abs_old!=merge_data.Amount_abs_OCP]

C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_17164\3905460538.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_OCP_work.counterpartyCode = data_OCP_work.counterpartyCode.astype(str)
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_17164\3905460538.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_OCP_work.ocpSegment = data_OCP_work.ocpSegment.fillna('External')
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_17164\3905460538.py:11: FutureWarning: The provided callable <built-in fun

In [32]:
old_data.shape[0], new_data.shape[0]

(35743, 35743)

In [25]:
old_data[old_data['Amount']!=0].shape[0], new_data[~new_data['volumeEffectiveCurrency'].isna()].shape[0]

(17706, 17706)

In [33]:
merge_data = is_approximately_equal_for_cols(merge_data,'Amount_abs_old','Amount_abs_OCP')

In [34]:
merge_data[merge_data['Diff__more_0.4']==True]

,holding,Comp_segment,Comp_code,Контрагент,currency,КалендГод/Месяц,Amount_abs_old,Версия_035,Версия_045,Версия_075,Версия_085,ocpSegment,buCode,counterpartyCode,effectiveCurrency,maturityDateTo,Amount_abs_OCP,Diff__more_0.4


In [28]:
assert 2==1

AssertionError: 

In [29]:
# to excel
Output_file = f'{str(date.today())}_ivest_vs_OCP.xlsx'
with pd.ExcelWriter(Output_file, engine='openpyxl') as writer:
    merge_data[merge_data['Diff__more_0.4']==True].to_excel(writer, index=False, sheet_name='compare')
    old_data.to_excel(writer, index=False, sheet_name='data_script')
    new_data.to_excel(writer, index=False, sheet_name='data_unOCP')


In [18]:
value = '1000153178'
new_data.loc[:,'buName':]
new_data[new_data.counterpartyCode==value]

,##batch_id,##deleted,##inserted,##origin_name,reportDate,rowType,sourceSystem,sourceTable,recordId,balanceItem,...,timeBucket,notionalCurrency,volumeNotionalCurrency,effectiveCurrency,effectiveCurrencyRate,volumeEffectiveCurrency,usdRate,volumeUsd,salesUnitsId,accountSubType
26090,41389909@WHWEEK,N,2024-10-25 12:12:50,WHFLEX,2024-09-30,D,ExternalBW,sapBwInvestmentReport,E.E2300002-10.04.010.07,Off-balance,...,<1M,INR,NaN,INR,None,NaN,0.011933,NaN,RUNWT,None
26115,41389909@WHWEEK,N,2024-10-25 12:12:50,WHFLEX,2024-09-30,D,ExternalBW,sapBwInvestmentReport,E.E2300002-10.04.020.02,Off-balance,...,<1M,INR,NaN,INR,None,NaN,0.011933,NaN,RUNWT,None
26182,41389909@WHWEEK,N,2024-10-25 12:12:50,WHFLEX,2024-09-30,D,ExternalBW,sapBwInvestmentReport,E.E2300002-10.04.030.69,Off-balance,...,<1M,INR,NaN,INR,None,NaN,0.011933,NaN,RUNWT,None
26183,41389909@WHWEEK,N,2024-10-25 12:12:50,WHFLEX,2024-09-30,D,ExternalBW,sapBwInvestmentReport,E.E2300002-10.04.030.70,Off-balance,...,<1M,INR,NaN,INR,None,NaN,0.011933,NaN,RUNWT,None
26184,41389909@WHWEEK,N,2024-10-25 12:12:50,WHFLEX,2024-09-30,D,ExternalBW,sapBwInvestmentReport,E.E2300002-10.04.030.71,Off-balance,...,<1M,INR,NaN,INR,None,NaN,0.011933,NaN,RUNWT,None
26185,41389909@WHWEEK,N,2024-10-25 12:12:50,WHFLEX,2024-09-30,D,ExternalBW,sapBwInvestmentReport,E.E2300002-10.04.030.72,Off-balance,...,<1M,INR,NaN,INR,None,NaN,0.011933,NaN,RUNWT,None
26186,41389909@WHWEEK,N,2024-10-25 12:12:50,WHFLEX,2024-09-30,D,ExternalBW,sapBwInvestmentReport,E.E2300002-10.04.030.74,Off-balance,...,<1M,INR,NaN,INR,None,NaN,0.011933,NaN,RUNWT,None
26191,41389909@WHWEEK,N,2024-10-25 12:12:50,WHFLEX,2024-09-30,D,ExternalBW,sapBwInvestmentReport,E.E2300002-10.04.030.73,Off-balance,...,<1M,INR,NaN,INR,None,NaN,0.011933,NaN,RUNWT,None
